In [138]:
import sys 
import os
import subprocess
import numpy as np 
import time
from itertools import product 

option = 1 
output_dir = '../output/run_clixo/option{}'.format(option); 
if not os.path.exists(output_dir): os.makedirs(output_dir)
stringdb_dir = '../output/extracting_string_interactions_subnetworks/'

In [135]:
# Obtaining the clixo program 
#clixo_script='/mnt/c/Users/Anubhav/Desktop/mhk7-clixo_0.3-0362bea/mhk7-clixo_0.3-0362bea/clixo'
clixo_script = '/gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo'

# Listing the string interactions file
confidence = 0.4
interactions_fn = os.path.join(stringdb_dir, 'synapse_inters_option_{}.tsv')
interactions_fn = interactions_fn.format(option)
interactions_fn 

'../output/extracting_string_interactions_subnetworks/synapse_inters_option_1.tsv'

## Code to run a single parameter set 


In [137]:
alpha = 0.21
beta = 0.71

# Naming the clixo output (can change if you need)
clixo_fn = os.path.join(output_dir, 
    'string_synapse_interactions_combined_score.clixo_alpha{}_beta{}.txt'.format(alpha, beta))
cmd = '{} {} {} {} > {}'.format(clixo_script, interactions_fn, alpha, beta, clixo_fn)
print('Running: {}'.format(cmd))

if not os.path.exists(clixo_fn): 
    sp = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    msgs = sp.communicate()
    print('\n'.join([x.decode() for x in msgs]))

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_1.tsv 0.21 0.71 > ../output/run_clixo/string_synapse_interactions_combined_score.clixo_alpha0.21_beta0.71.txt




### Testing different values of alpha and beta

In [98]:
def get_alpha_beta(l):
    """
    Yield function to get the correct alpha and beta names.
    
    """
    for x in l: 
        alpha = '{:.1f}'.format(x[0])
        beta =  '{:.1f}'.format(x[1])
        yield((alpha, beta))
        
    yield(None, None)

In [120]:
# Gathering all the 
parameters = product(np.arange(0, 1.1, 0.1), np.arange(0, 1.1, 0.1))
parameters = get_alpha_beta(parameters)

jobs = {}

In [ ]:
concurrent_jobs = 10
num_loops = 0 
while True: 
    
    # Get the status of jobs  
    jobs = {job: job.poll() for job in jobs}
            
    # Remove jobs
    job_keys = jobs.keys()
    for job_key in list(job_keys):  
        status = jobs[job_key]  
        # Job is no longer running, remove it 
        if status != None: 
            jobs.pop(job_key)      
            
    # Start new jobs 
    num_new_jobs = concurrent_jobs - len(jobs)  
    
    #print('num_new_jobs', num_new_jobs)
    
    for i in range(num_new_jobs):
            
        # Pulling the alpha and beta values 
        alpha, beta = next(parameters)
        #print('alpha, beta', alpha, beta)
            
        # Checking whether there are more parameters to run  
        if alpha == None: 
            break 
            
        # Running CliXo jobs with a new set of parameters
        clixo_fn = 'string_synapse_interactions_combined_score'
        clixo_fn += '.clixo_alpha{}_beta{}.txt'
        clixo_fn = clixo_fn.format(alpha, beta)
        clixo_fn = os.path.join(output_dir, clixo_fn)

        if not os.path.exists(clixo_fn):
            cmd = '{} {} {} {} > {}'.format(clixo_script, 
                                            interactions_fn, 
                                            alpha, 
                                            beta, 
                                            clixo_fn)
            print('Running: {}'.format(cmd))
            job = subprocess.Popen(cmd, 
                                   stderr=subprocess.PIPE, 
                                   stdout=subprocess.PIPE, 
                                   shell=True)

            # Add the status of the job as running (=None)
            jobs[job] = None 
            
    if len(jobs) == concurrent_jobs: 
        time.sleep(300)
    else:
        time.sleep(5)
        
    print('Iteraction number: {}, number of jobs running: {}'.\
          format(num_loops, len(jobs)))
    num_loops+= 1

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions/string_synapse_interactions.tsv 0.0 1.0 > ../output/run_clixo/string_synapse_interactions_combined_score.clixo_alpha0.0_beta1.0.txt
Iteraction number: 0, number of jobs running: 10
Iteraction number: 1, number of jobs running: 10
Iteraction number: 2, number of jobs running: 10
Iteraction number: 3, number of jobs running: 10
Iteraction number: 4, number of jobs running: 10
Iteraction number: 5, number of jobs running: 10
Iteraction number: 6, number of jobs running: 10
Iteraction number: 7, number of jobs running: 10
Iteraction number: 8, number of jobs running: 10
Iteraction number: 9, number of jobs running: 10
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions/string_synapse_interactions.tsv 0.1 0.0 > ../output/run_clixo/string_synapse_interactions_combined_score.clixo_alpha0.1_beta0.0.txt
Iteraction

In [ ]:
2 + 2 